## Day 81 Lecture 2 Assignment

In this assignment, we will learn about Dask dataframes. We will use the census dataset loaded below and perform some exploratory data analysis.

In [79]:
!pip install dask[complete] --quiet 
!pip install aiohttp --quiet

In [80]:
import dask.dataframe as dd
import numpy as np

In [81]:
census = dd.read_csv('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/Data%20Sets%20Big%20Data/acs2015_census_tract_data.csv')

In [82]:
census.head()

,CensusTract,State,County,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,Pacific,Citizen,Income,IncomeErr,IncomePerCap,IncomePerCapErr,Poverty,ChildPoverty,Professional,Service,Office,Construction,Production,Drive,Carpool,Transit,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment
0,1001020100,Alabama,Autauga,1948,940,1008,0.9,87.4,7.7,0.3,0.6,0.0,1503,61838.0,11900.0,25713.0,4548.0,8.1,8.4,34.7,17.0,21.3,11.9,15.2,90.2,4.8,0.0,0.5,2.3,2.1,25.0,943,77.1,18.3,4.6,0.0,5.4
1,1001020200,Alabama,Autauga,2156,1059,1097,0.8,40.4,53.3,0.0,2.3,0.0,1662,32303.0,13538.0,18021.0,2474.0,25.5,40.3,22.3,24.7,21.5,9.4,22.0,86.3,13.1,0.0,0.0,0.7,0.0,23.4,753,77.0,16.9,6.1,0.0,13.3
2,1001020300,Alabama,Autauga,2968,1364,1604,0.0,74.5,18.6,0.5,1.4,0.3,2335,44922.0,5629.0,20689.0,2817.0,12.7,19.7,31.4,24.9,22.1,9.2,12.4,94.8,2.8,0.0,0.0,0.0,2.5,19.6,1373,64.1,23.6,12.3,0.0,6.2
3,1001020400,Alabama,Autauga,4423,2172,2251,10.5,82.8,3.7,1.6,0.0,0.0,3306,54329.0,7003.0,24125.0,2870.0,2.1,1.6,27.0,20.8,27.0,8.7,16.4,86.6,9.1,0.0,0.0,2.6,1.6,25.3,1782,75.7,21.2,3.1,0.0,10.8
4,1001020500,Alabama,Autauga,10763,4922,5841,0.7,68.5,24.8,0.0,3.8,0.0,7666,51965.0,6935.0,27526.0,2813.0,11.4,17.5,49.6,14.2,18.2,2.1,15.8,88.0,10.5,0.0,0.0,0.6,0.9,24.8,5037,67.1,27.6,5.3,0.0,4.2


The dataset described above contains the aggregated data for each county in the United States. For example, the number in the Carpool column describes the percent of people in the county who carpool to work.

Print all the column names in the dataset and describe the data in each column

In [83]:
census.columns

Index(['CensusTract', 'State', 'County', 'TotalPop', 'Men', 'Women',
       'Hispanic', 'White', 'Black', 'Native', 'Asian', 'Pacific', 'Citizen',
       'Income', 'IncomeErr', 'IncomePerCap', 'IncomePerCapErr', 'Poverty',
       'ChildPoverty', 'Professional', 'Service', 'Office', 'Construction',
       'Production', 'Drive', 'Carpool', 'Transit', 'Walk', 'OtherTransp',
       'WorkAtHome', 'MeanCommute', 'Employed', 'PrivateWork', 'PublicWork',
       'SelfEmployed', 'FamilyWork', 'Unemployment'],
      dtype='object')

In [84]:
census.describe().compute()

,CensusTract,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,Pacific,Citizen,Income,IncomeErr,IncomePerCap,IncomePerCapErr,Poverty,ChildPoverty,Professional,Service,Office,Construction,Production,Drive,Carpool,Transit,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment
count,7.400100e+04,74001.000000,74001.000000,74001.000000,73311.000000,73311.000000,73311.000000,73311.000000,73311.000000,73311.000000,74001.000000,72901.000000,72901.000000,73261.000000,73261.000000,73166.000000,72883.000000,73194.000000,73194.000000,73194.000000,73194.000000,73194.000000,73204.000000,73204.000000,73204.000000,73204.000000,73204.000000,73204.000000,73052.000000,74001.000000,73194.000000,73194.000000,73194.000000,73194.000000,73199.000000
mean,2.839113e+10,4325.591465,2127.648816,2197.942649,16.862810,62.032106,13.272581,0.727726,4.588253,0.145012,3043.079566,57225.564862,9134.493848,28491.234299,3942.909229,16.958020,22.486827,34.798843,19.101381,23.951559,9.292337,12.856005,75.533887,9.627468,5.456050,3.123340,1.891606,4.368093,25.667357,1983.907366,78.975238,14.621566,6.233814,0.169772,9.028663
std,1.647593e+10,2129.306903,1072.332031,1095.730931,22.940695,30.684152,21.762483,4.488340,8.794003,1.037354,1475.492946,28663.331962,5920.336551,15047.068660,3023.027898,13.196518,19.190909,15.007075,8.279143,5.957279,6.023290,7.671588,15.371620,5.367321,11.724287,5.881237,2.596198,3.904990,6.964881,1073.429808,8.345758,7.535786,4.042990,0.458227,5.955441
min,1.001020e+09,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2611.000000,390.000000,128.000000,85.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.303901e+10,2891.000000,1409.000000,1461.000000,2.400000,39.400000,0.700000,0.000000,0.200000,0.000000,2037.000000,37683.000000,5317.000000,19123.000000,2312.000000,7.200000,7.000000,24.100000,13.400000,20.100000,5.000000,7.100000,72.000000,6.000000,0.000000,0.400000,0.400000,1.800000,20.800000,1249.000000,74.600000,9.600000,3.500000,0.000000,5.100000
50%,2.804700e+10,4063.000000,1986.000000,2066.000000,7.000000,71.400000,3.700000,0.000000,1.400000,0.000000,2863.000000,51094.000000,7732.000000,25344.000000,3127.000000,13.400000,17.800000,32.600000,17.900000,23.800000,8.400000,11.800000,79.700000,8.800000,1.100000,1.400000,1.100000,3.500000,25.000000,1846.000000,80.100000,13.400000,5.500000,0.000000,7.700000
75%,4.200341e+10,5442.000000,2674.000000,2774.000000,20.400000,88.300000,14.400000,0.400000,4.800000,0.000000,3838.000000,70117.000000,11258.000000,33894.000000,4537.000000,23.100000,33.800000,43.875000,23.600000,27.500000,12.500000,17.400000,84.900000,12.300000,4.700000,3.500000,2.500000,5.900000,29.800000,2553.000000,84.600000,18.200000,8.100000,0.000000,11.400000
max,7.215375e+10,53812.000000,27962.000000,27250.000000,100.000000,100.000000,100.000000,100.000000,91.300000,84.700000,37416.000000,248750.000000,123116.000000,254204.000000,134380.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,80.000000,24075.000000,100.000000,100.000000,100.000000,26.500000,100.000000


How many counties have a unemployement rate above 10%?

In [85]:
np.count_nonzero(census.groupby("County").mean()["Unemployment"].compute().values>10)

533

There are 533 counties that have an unemployment rate above 10%.

Print the number of counties in each state.

In [87]:
census.groupby("State")["County"].nunique().compute()

State
Alabama                  67
Alaska                   29
Arizona                  15
Arkansas                 75
California               58
Colorado                 64
Connecticut               8
Delaware                  3
District of Columbia      1
Florida                  67
Georgia                 159
Hawaii                    5
Idaho                    44
Illinois                102
Indiana                  92
Iowa                     99
Kansas                  105
Kentucky                120
Louisiana                64
Maine                    16
Maryland                 24
Massachusetts            14
Michigan                 83
Minnesota                87
Mississippi              82
Missouri                115
Montana                  56
Nebraska                 93
Nevada                   17
New Hampshire            10
New Jersey               21
New Mexico               33
New York                 62
North Carolina          100
North Dakota             53
Ohio          

Add a high child poverty boolean column. This column will be 1 if the rate of child poverty is over %20 and 0 otherwise.

In [90]:
census['HighChildPoverty'] = census['ChildPoverty'].map(lambda x: 1 if x>20 else 0)

census.compute().head(10)

,CensusTract,State,County,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,Pacific,Citizen,Income,IncomeErr,IncomePerCap,IncomePerCapErr,Poverty,ChildPoverty,Professional,Service,Office,Construction,Production,Drive,Carpool,Transit,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment,HighChildPoverty
0,1001020100,Alabama,Autauga,1948,940,1008,0.9,87.4,7.7,0.3,0.6,0.0,1503,61838.0,11900.0,25713.0,4548.0,8.1,8.4,34.7,17.0,21.3,11.9,15.2,90.2,4.8,0.0,0.5,2.3,2.1,25.0,943,77.1,18.3,4.6,0.0,5.4,0
1,1001020200,Alabama,Autauga,2156,1059,1097,0.8,40.4,53.3,0.0,2.3,0.0,1662,32303.0,13538.0,18021.0,2474.0,25.5,40.3,22.3,24.7,21.5,9.4,22.0,86.3,13.1,0.0,0.0,0.7,0.0,23.4,753,77.0,16.9,6.1,0.0,13.3,1
2,1001020300,Alabama,Autauga,2968,1364,1604,0.0,74.5,18.6,0.5,1.4,0.3,2335,44922.0,5629.0,20689.0,2817.0,12.7,19.7,31.4,24.9,22.1,9.2,12.4,94.8,2.8,0.0,0.0,0.0,2.5,19.6,1373,64.1,23.6,12.3,0.0,6.2,0
3,1001020400,Alabama,Autauga,4423,2172,2251,10.5,82.8,3.7,1.6,0.0,0.0,3306,54329.0,7003.0,24125.0,2870.0,2.1,1.6,27.0,20.8,27.0,8.7,16.4,86.6,9.1,0.0,0.0,2.6,1.6,25.3,1782,75.7,21.2,3.1,0.0,10.8,0
4,1001020500,Alabama,Autauga,10763,4922,5841,0.7,68.5,24.8,0.0,3.8,0.0,7666,51965.0,6935.0,27526.0,2813.0,11.4,17.5,49.6,14.2,18.2,2.1,15.8,88.0,10.5,0.0,0.0,0.6,0.9,24.8,5037,67.1,27.6,5.3,0.0,4.2,0
5,1001020600,Alabama,Autauga,3851,1787,2064,13.1,72.9,11.9,0.0,0.0,0.0,2642,63092.0,9585.0,30480.0,7550.0,14.4,21.9,24.2,17.5,35.4,7.9,14.9,82.7,6.9,0.0,0.0,6.0,4.5,19.8,1560,79.4,14.7,5.8,0.0,10.9,1
6,1001020700,Alabama,Autauga,2761,1210,1551,3.8,74.5,19.7,0.0,0.0,0.0,2060,34821.0,7867.0,20442.0,3245.0,28.9,41.9,19.5,29.6,25.3,10.1,15.5,92.4,7.6,0.0,0.0,0.0,0.0,20.0,1166,82.0,14.6,3.4,0.0,11.4,1
7,1001020801,Alabama,Autauga,3187,1502,1685,1.3,84.0,10.7,3.1,0.0,0.0,2391,73728.0,2447.0,32813.0,4669.0,13.0,25.9,42.8,10.7,34.2,5.5,6.8,84.3,8.1,0.0,0.0,1.7,5.9,24.3,1502,78.1,14.8,7.1,0.0,8.2,1
8,1001020802,Alabama,Autauga,10915,5486,5429,1.4,89.5,8.4,0.0,0.0,0.0,7778,60063.0,8602.0,24028.0,2233.0,13.9,18.3,31.5,17.5,26.1,7.8,17.1,90.1,8.6,0.0,0.0,0.0,1.3,29.4,4348,73.3,22.1,4.6,0.0,8.7,0
9,1001020900,Alabama,Autauga,5668,2897,2771,0.4,85.5,12.1,0.0,0.3,0.0,4217,41287.0,7857.0,24710.0,4149.0,6.8,10.0,29.3,13.7,17.7,11.0,28.3,88.7,7.9,0.0,0.0,1.2,2.1,32.9,2485,77.9,15.2,6.9,0.0,7.2,0


What are the top 10 most populated counties?

In [91]:
census.groupby("County").sum()["TotalPop"].compute().sort_values(ascending=False).head(10)

County
Los Angeles    10038388
Cook            5258620
Orange          5025603
Harris          4389138
Maricopa        4018143
Montgomery      3635047
Jefferson       3623682
San Diego       3223096
Clark           2852761
Kings           2746257
Name: TotalPop, dtype: int64

What is the current number of partitions? 

In [92]:
census.npartitions

1

As it stands, since I'm still using the default init params, I only have 1 partition. It is as if I'm still using Pandas DataFrame.